# Loading

In [ ]:
!pip install scanpy
!pip install episcanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 11.2 MB 44.8 MB/s 
     |████████████████████████████████| 88 kB 6.3 MB/s 
     |████████████████████████████████| 96 kB 4.7 MB/s 
     |████████████████████████████████| 965 kB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=727fbe26c4ae496a1b8fc15ff6799fe40044484543fdcfc13fcff3058c367dfd
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=6c3d5ca8e1f7eb0ba74b75f32dc341c2732d74f7b26cecd87554c760d74baa30
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50.3 MB 1.2 MB/s 
     |████████████████████████████████| 184 kB 53.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.0 MB 36.7 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 15.4 MB 41.1 MB/s 
  Created wheel for episcanpy: filename=episcanpy-0.3.2-py3-none-any.whl size=125232 sha256=8f38276f50bfbc595f77842758077c6a9e33f8f9dafe8f06bd08d4a02027fd2b
  Stored in directory: /root/.cache/pip/wheels/80/ee/b2/c1d719a7519d3eb034a7959d6bb24ac5603a783c75b45c8b12
  Created wheel for bamnostic: filename=bamnostic-1.1.8-py3-none-any.whl size=178413 sha256=e826e4980fdb2fb5b93653e805d1be24d0ddd2bc65d9693172c945651e2dd099
  Stored in directory: /root/.cache/pip/wheels/14/c1/83/8e29a56af

In [ ]:
import anndata
import networkx as nx
import scanpy as sc
#import scglue
from matplotlib import rcParams
import pandas as pd
import numpy as np
import anndata as ad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/cancer_water/GSE158398_P2003139.h5ad /content
!cp /content/drive/MyDrive/cancer_water/GSE158398_P2003138.h5ad /content

In [ ]:
!pip install episcanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Function and basic test

In [ ]:
import anndata
import scanpy as sc
import episcanpy as epi

atac1=anndata.read_h5ad("GSE158398_P2003138.h5ad")
atac2=anndata.read_h5ad("GSE158398_P2003139.h5ad")

In [ ]:
import anndata
import networkx as nx
import scanpy as sc
#import scglue
from matplotlib import rcParams
import pandas as pd
import numpy as np
import anndata as ad

In [ ]:
import pandas as pd
import numpy as np
import anndata as ad

def atac_concat_get_index(adata1,adata2):
  import gc
  
  # 数据预处理
  adata1.var.loc[:,'ATAC'] = adata1.var.index
  adata1.var = pd.DataFrame((x.split('_') for x in adata1.var['ATAC']) , index = adata1.var.index , columns = ['chr','chromStart','chromEnd'])
  adata2.var.loc[:,'ATAC'] = adata2.var.index
  adata2.var = pd.DataFrame((x.split('_') for x in adata2.var['ATAC']) , index = adata2.var.index , columns = ['chr','chromStart','chromEnd'])

  # 判断是否出现 不存在某条染色体的情况，如果存在，我们希望染色体较少的一方为adata1（因为后续的分析都是针对adata1为染色体较少的情况进行分析的）
  # 如果出现adata1的染色体比adata2的染色体多，则调换位置
  if len(set(adata1.var.chr))>len(set(adata2.var.chr)):
    adata_chr = adata1
    adata1 = adata2
    adata2 = adata_chr
    Exchange = True #Exchange用于判断调换位置的特殊情况
  else:
    Exchange = False
 

  # 循环函数确定和拆分
  atac_var = adata1.var
  Promoter = adata2.var
  Promoter.loc[:,'chr']=Promoter.loc[:,'chr']+'_'
  Promoter.loc[:,'index_1'] = Promoter.index
  atac_var.loc[:,'chr']=atac_var.loc[:,'chr']+'_'
  atac_var.loc[:,'index_1'] = atac_var.index
  chr	= set(Promoter.chr)

  # 比对函数开始
  ATAC = {}
  PROMOTER = {}
  for x in chr:
    # 染色体拆分
    test = pd.DataFrame(columns=['Length'])
    ATAC[x] = test
    ATAC[x]=atac_var[atac_var.index.str.contains(x)]
    PROMOTER[x] = test
    test = pd.DataFrame(columns=['Length'])
    PROMOTER[x]=Promoter.loc[Promoter['chr'] == x]

    # 输出循环染色体
    print('start %s' % (x))
  
    # 特殊情况处理-判断该条染色体是否为空值,并进行赋值
    if len(ATAC[x].index)==0:
      ATAC[x] = PROMOTER[x]
      ATAC_INDEX = ATAC[x].columns.to_list() + ['overleaf_1','overleaf_2','chromStart_New','chromEnd_New','index_2']
      ATAC[x] = ATAC[x].reindex(columns=ATAC_INDEX, fill_value='retain')
      ATAC[x].loc[:,'index_2'] = ATAC[x].loc[:,'index_1']
      continue
    

    # index赋值
    ATAC_INDEX = ATAC[x].columns.to_list() + ['overleaf_1','overleaf_2','chromStart_New','chromEnd_New','index_2']
    ATAC[x] = ATAC[x].reindex(columns=ATAC_INDEX, fill_value='')
    
    # 函数主循环体
    for i in range(0,len(PROMOTER[x].chromEnd)):
      left = 0
      right = len(ATAC[x].chromEnd)-1
      target_Start = int(PROMOTER[x].chromStart.iloc[i])
      target_End = int(PROMOTER[x].chromEnd.iloc[i])


      n=0
      while(n<30):
        # 中位数表示 #
        midIndex = round(int((left+right)/2))
        midValue_Start = int(ATAC[x].chromStart.iloc[midIndex])
        midValue_End = int(ATAC[x].chromEnd.iloc[midIndex])
        # 三次condition判断 #

        if (target_Start > midValue_End) :
          left = midIndex
          n=n+1

        elif (target_End < midValue_Start):
          right = midIndex
          n=n+1
        else:
          Length = target_End - target_Start
          ATAC[x].loc[:,'overleaf_1'].iloc[midIndex] = (target_Start - midValue_End)/Length
          ATAC[x].loc[:,'overleaf_2'].iloc[midIndex] = (midValue_Start - target_End)/Length
          list = [midValue_Start,midValue_End,target_Start,target_End]
          ATAC[x].loc[:,'chromStart_New'].iloc[midIndex] = min(list)
          ATAC[x].loc[:,'chromEnd_New'].iloc[midIndex] = max(list)
          ATAC[x].loc[:,'index_2'].iloc[midIndex] = PROMOTER[x].index[i]
          break
 
  # 拼接dataframe
  ATAC_Promoter = pd.DataFrame()
  for i in chr:
    ATAC_Promoter = pd.concat([ATAC_Promoter,ATAC[i],],axis=0)
    
  # 删除未比对上序列
  ATAC_Promoter = ATAC_Promoter[ATAC_Promoter['overleaf_1']!='']

  # 定义新序列拼接函数
  def indeX(series):
    chr = series["chr"]
    chromStart_New = series["chromStart_New"]
    chromEnd_New = series["chromEnd_New"]
    indeX = chr + str(chromStart_New) +'_' +str(chromEnd_New)
    return indeX

  # 拼接新序列
  ATAC_Promoter.loc[:,'index_new'] = ''
  ATAC_Promoter.loc[:,'index_new'] = ATAC_Promoter.apply(indeX,axis=1)

  # 提取新序列
  pair_index = pd.DataFrame()
  pair_index.loc[:,'overleaf_1'] = ATAC_Promoter.loc[:,'overleaf_1']
  pair_index.loc[:,'overleaf_2'] = ATAC_Promoter.loc[:,'overleaf_2']
  pair_index.loc[:,'index_new'] = ATAC_Promoter.loc[:,'index_new']
  # 若出现adata1调换adata2的特殊情况，则在这里把adata1和adata2再调换回来
  if Exchange==True:
    pair_index.loc[:,'index_1'] = ATAC_Promoter.loc[:,'index_2']
    pair_index.loc[:,'index_2'] = ATAC_Promoter.loc[:,'index_1']
  else:
    pair_index.loc[:,'index_1'] = ATAC_Promoter.loc[:,'index_1']
    pair_index.loc[:,'index_2'] = ATAC_Promoter.loc[:,'index_2']
  # 特殊情况处理-空值染色体
  pair_index.loc[pair_index['overleaf_1']=='retain','index_new'] = pair_index.loc[pair_index['overleaf_1']=='retain','index_1']
  gc.collect()
  return pair_index


def atac_concat_inner(adata1,adata2,pair_index):
  import gc
  
  # 设置基本参数
  adata1.var.loc[:,'ATAC'] = adata1.var.index
  df1 = pd.DataFrame((x.split('_') for x in adata1.var['ATAC']) , index = adata1.var.index , columns = ['chr','chromStart','chromEnd'])
  adata1.var = df1
  adata2.var.loc[:,'ATAC'] = adata2.var.index
  df2 = pd.DataFrame((x.split('_') for x in adata2.var['ATAC']) , index = adata2.var.index , columns = ['chr','chromStart','chromEnd'])
  adata2.var = df2

  # 判断是否存在需要调换位置的情况
  if len(set(adata1.var.chr))>len(set(adata2.var.chr)): 
    Exchange = True #Exchange用于判断调换位置的特殊情况
  else:
    Exchange = False
  
  # 通过overleaf_1的赋值来判断是否存在缺失染色体的情况
  if Exchange==False:
    atac1_new = adata1[:,pair_index[pair_index['overleaf_1']!='retain'].index_1]
    atac1_new.var = pair_index[pair_index['overleaf_1']!='retain']
    atac1_new.var.index = atac1_new.var.index_new
    atac2_new = adata2[:,pair_index.index_2]
    atac2_new.var = pair_index
    atac2_new.var.index = atac2_new.var.index_new

  if Exchange==True:
    atac1_new = adata1[:,pair_index.index_1]
    atac1_new.var = pair_index
    atac1_new.var.index = atac1_new.var.index_new
    atac2_new = adata2[:,pair_index[pair_index['overleaf_1']!='retain'].index_2]
    atac2_new.var = pair_index[pair_index['overleaf_1']!='retain']
    atac2_new.var.index = atac2_new.var.index_new
  
  # 配对
  adata_pair = ad.concat([atac1_new,atac2_new],axis=0,join='outer',fill_value=0)
  adata_pair.obs_names_make_unique()
  adata_pair
  gc.collect()
  return adata_pair

def atac_concat_outer(adata1,adata2,pair_index):
  import gc
  
    # 提取关键参数
  adata1.var.loc[:,'ATAC'] = adata1.var.index
  adata1.var = pd.DataFrame((x.split('_') for x in adata1.var['ATAC']) , index = adata1.var.index , columns = ['chr','chromStart','chromEnd'])
  adata2.var.loc[:,'ATAC'] = adata2.var.index
  adata2.var = pd.DataFrame((x.split('_') for x in adata2.var['ATAC']) , index = adata2.var.index , columns = ['chr','chromStart','chromEnd'])

    # 判断是否存在需要调换位置的情况
  if len(set(adata1.var.chr))>len(set(adata2.var.chr)): 
    Exchange = True #Exchange用于判断调换位置的特殊情况
  else:
    Exchange = False
  
    # 通过overleaf_1的赋值来判断是否存在缺失染色体的情况
  if Exchange==False:
    atac1_new = adata1[:,pair_index[pair_index['overleaf_1']!='retain'].index_1]
    atac1_new.var = pair_index[pair_index['overleaf_1']!='retain']
    atac1_new.var.index = atac1_new.var.index_new
    atac2_new = adata2[:,pair_index.index_2]
    atac2_new.var = pair_index
    atac2_new.var.index = atac2_new.var.index_new

  if Exchange==True:
    atac1_new = adata1[:,pair_index.index_1]
    atac1_new.var = pair_index
    atac1_new.var.index = atac1_new.var.index_new
    atac2_new = adata2[:,pair_index[pair_index['overleaf_1']!='retain'].index_2]
    atac2_new.var = pair_index[pair_index['overleaf_1']!='retain']
    atac2_new.var.index = atac2_new.var.index_new

  # 计算前置数据
  index_list = atac1_new.var
  index_list.index = index_list.index_1
  df_month = pd.DataFrame(columns=['name'])
  df_month.name = adata1.var.index.values
  df_month.index = df_month.name.values
  df_month.drop(['name'],axis=1,inplace=True)
  # 整合index
  var_1 = pd.concat([index_list,df_month],join='outer',axis=1).copy()
  var_1 = var_1.fillna('*')
  var_1['name']=var_1.index
  # 替换index
  var_1.loc[var_1['index_1']!='*','name'] = var_1[var_1['index_1']!='*'].index_new.values
  var_1.index = var_1.name
  adata1.var =var_1
  # 输出结果
  adata1.var_name = var_1.index
  
   # 同上
  index_list = atac2_new.var
  index_list.index = index_list.index_2
  df_month = pd.DataFrame(columns=['name'])
  df_month.name = adata2.var.index.values
  df_month.index = df_month.name.values
  df_month.drop(['name'],axis=1,inplace=True)
  var_2 = pd.concat([index_list,df_month],join='outer',axis=1).copy()
  var_2 = var_2.fillna('*')
  var_2['name']=var_2.index
  var_2.loc[var_2['index_1']!='*','name'] = var_2[var_2['index_1']!='*'].index_new.values
  var_2.index = var_2.name
  adata2.var =var_2
  adata2.var_name = var_2.index

  adata_pair = ad.concat([adata1,adata2],axis=0,join='outer',fill_value=0)
  adata_pair.obs_names_make_unique()
  adata_pair

  gc.collect()
  return adata_pair

In [ ]:
pair_index = atac_concat_get_index(atac1,atac2)
pair_index

start chr11_
start chr1_
start chr21_
start chr22_
start chr20_
start chrX_
start chr3_
start chrY_
start chr4_
start chr16_
start chr12_
start chr7_
start chr10_
start chr17_
start chr13_
start chr18_
start chr8_
start chr15_
start chr19_
start chr9_
start chr5_
start chr14_
start chr2_
start chr6_


,overleaf_1,overleaf_2,index_new,index_1,index_2
chr11_192218_192324,-0.775561,-0.488778,chr11_192013_192414,chr11_192218_192324,chr11_192013_192414
chr11_202034_202344,-0.754128,-0.814679,chr11_201933_202478,chr11_202034_202344,chr11_201933_202478
chr11_206300_209548,-0.910515,-0.997763,chr11_206292_209868,chr11_206300_209548,chr11_206292_209868
chr11_235591_237610,-0.906918,-0.939623,chr11_235447_237832,chr11_235591_237610,chr11_235447_237832
chr11_250668_251199,-0.821958,-0.965875,chr11_250645_251319,chr11_250668_251199,chr11_250645_251319
...,...,...,...,...,...
chr6_170459476_170459849,-0.906522,-0.904348,chr6_170459432_170459892,chr6_170459476_170459849,chr6_170459432_170459892
chr6_170476497_170476505,-0.292453,-0.726415,chr6_170476381_170476805,chr6_170476497_170476505,chr6_170476381_170476805
chr6_170516920_170517437,-0.975962,-0.852564,chr6_170516828_170517452,chr6_170516920_170517437,chr6_170516828_170517452
chr6_170529778_170530236,-0.995833,-0.958333,chr6_170529758_170530238,chr6_170529778_170530236,chr6_170529758_170530238


In [ ]:
adata_inner = atac_concat_inner(atac1,atac2,pair_index) #交集
adata_inner

AnnData object with n_obs × n_vars = 12844 × 71265

In [ ]:
adata_outer = atac_concat_outer(atac1,atac2,pair_index) #并集
adata_outer

AnnData object with n_obs × n_vars = 12844 × 93436

# 函数功能性测试

In [ ]:
def Overlap_inner(adata1,adata2,adata_inner):
  atac1 = adata1
  atac2 = adata2

  # 提取关键参数
  adata_inner.var.loc[:,'ATAC'] = adata_inner.var.index
  adata_inner.var = pd.DataFrame((x.split('_') for x in adata_inner.var['ATAC']) , index = adata_inner.var.index , columns = ['chr','chromStart','chromEnd'])
  # 提取关键参数
  atac1.var.loc[:,'ATAC'] = atac1.var.index
  atac1.var = pd.DataFrame((x.split('_') for x in atac1.var['ATAC']) , index = atac1.var.index , columns = ['chr','chromStart','chromEnd'])
  # 提取关键参数
  atac2.var.loc[:,'ATAC'] = atac2.var.index
  atac2.var = pd.DataFrame((x.split('_') for x in atac2.var['ATAC']) , index = atac2.var.index , columns = ['chr','chromStart','chromEnd'])

  atac_var = adata_inner.var
  atac_1_var = atac1.var
  atac_2_var = atac2.var
  chr	= set(atac_var.chr)

  # 设置重叠率存储变量
  Overlap = pd.DataFrame()
  OverlapIndex = Overlap.columns.to_list() + ['ChromLength_atac1','ChromLength_atac2','ChromLength_atac_inner','Overlap_atac1','Overlap_atac2']
  Overlap = Overlap.reindex(columns=OverlapIndex, fill_value=0)
  # 重叠率计算函数开始
  ATAC_INNER = {}
  ATAC_1 = {}
  ATAC_2 = {}
  for x in chr:
    # 染色体拆分
    test = pd.DataFrame(columns=['Length'])
    ATAC_INNER[x] = test
    ATAC_INNER[x]=atac_var[atac_var.chr == x]
    ATAC_INNER[x].loc[:,'length'] = ''
    test = pd.DataFrame(columns=['Length'])
    ATAC_1[x] = test
    ATAC_1[x]=atac_1_var[atac_1_var.chr == x]
    ATAC_1[x].loc[:,'length'] = ''

    test = pd.DataFrame(columns=['Length'])
    ATAC_2[x] = test
    ATAC_2[x]=atac_2_var[atac_2_var.chr == x]
    ATAC_2[x].loc[:,'length'] = ''

    # 输出标记
    print('start %s' % x)
  
    # 计算宽度
    Overlap.loc[x,'ChromLength_atac_inner'] = (ATAC_INNER[x].to_numpy()[:,[2]].astype(int) - ATAC_INNER[x].to_numpy()[:,[1]].astype(int)).sum()
    Overlap.loc[x,'ChromLength_atac1'] = (ATAC_1[x].to_numpy()[:,[2]].astype(int) - ATAC_1[x].to_numpy()[:,[1]].astype(int)).sum()
    Overlap.loc[x,'ChromLength_atac2'] = (ATAC_2[x].to_numpy()[:,[2]].astype(int) - ATAC_2[x].to_numpy()[:,[1]].astype(int)).sum()

  # 计算比例
  Overlap.loc[:,'Overlap_atac1'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac1'].values
  Overlap.loc[:,'Overlap_atac2'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac2'].values
  
  return Overlap


In [ ]:
Overlap = Overlap_inner(atac1,atac2,adata_inner)
Overlap

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


start chr11
start chr4
start chr17
start chr10
start chr3
start chr22
start chr20
start chr1
start chrY
start chr18
start chr8
start chr12
start chr19
start chr15
start chr13
start chrX
start chr2
start chr5
start chr21
start chr6
start chr14
start chr7
start chr16
start chr9


,ChromLength_atac1,ChromLength_atac2,ChromLength_atac_inner,Overlap_atac1,Overlap_atac2
chr11,4938000.0,4921961.0,4642162.0,0.940090,0.943153
chr4,3293066.0,3344953.0,3152547.0,0.957329,0.942479
chr17,5163841.0,5111292.0,4851856.0,0.939583,0.949243
chr10,4100583.0,4159248.0,3914269.0,0.954564,0.941100
chr3,5421467.0,5486020.0,5166697.0,0.953007,0.941793
chr22,2316963.0,2284808.0,2182849.0,0.942116,0.955375
chr20,2554597.0,2584346.0,2426533.0,0.949869,0.938935
chr1,9036173.0,9081936.0,8620606.0,0.954011,0.949204
chrY,21584.0,14996.0,14590.0,0.675964,0.972926
chr18,1585194.0,1600241.0,1511095.0,0.953256,0.944292


In [ ]:
def Overlap_inner(adata1,adata2,adata_inner):
  atac1 = adata1
  atac2 = adata2

  # 提取关键参数
  adata_inner.var.loc[:,'ATAC'] = adata_inner.var.index
  adata_inner.var = pd.DataFrame((x.split('_') for x in adata_inner.var['ATAC']) , index = adata_inner.var.index , columns = ['chr','chromStart','chromEnd'])
  # 提取关键参数
  atac1.var.loc[:,'ATAC'] = atac1.var.index
  atac1.var = pd.DataFrame((x.split('_') for x in atac1.var['ATAC']) , index = atac1.var.index , columns = ['chr','chromStart','chromEnd'])
  # 提取关键参数
  atac2.var.loc[:,'ATAC'] = atac2.var.index
  atac2.var = pd.DataFrame((x.split('_') for x in atac2.var['ATAC']) , index = atac2.var.index , columns = ['chr','chromStart','chromEnd'])

  atac_var = adata_inner.var
  atac_1_var = atac1.var
  atac_2_var = atac2.var
  chr	= set(atac_var.chr)

  # 设置重叠率存储变量
  Overlap = pd.DataFrame()
  OverlapIndex = Overlap.columns.to_list() + ['ChromLength_atac1','ChromLength_atac2','ChromLength_atac_inner','Overlap_atac1','Overlap_atac2']
  Overlap = Overlap.reindex(columns=OverlapIndex, fill_value=0)
  # 重叠率计算函数开始
  ATAC_INNER = {}
  ATAC_1 = {}
  ATAC_2 = {}
  for x in chr:
    # 染色体拆分
    test = pd.DataFrame(columns=['Length'])
    ATAC_INNER[x] = test
    ATAC_INNER[x]=atac_var[atac_var.chr == x]
    ATAC_INNER[x].loc[:,'length'] = ''
    test = pd.DataFrame(columns=['Length'])
    ATAC_1[x] = test
    ATAC_1[x]=atac_1_var[atac_1_var.chr == x]
    ATAC_1[x].loc[:,'length'] = ''

    test = pd.DataFrame(columns=['Length'])
    ATAC_2[x] = test
    ATAC_2[x]=atac_2_var[atac_2_var.chr == x]
    ATAC_2[x].loc[:,'length'] = ''

    # 输出标记
    print('start %s' % x)
  
    # 计算宽度
    Overlap.loc[x,'ChromLength_atac_inner'] = (ATAC_INNER[x].to_numpy()[:,[2]].astype(int) - ATAC_INNER[x].to_numpy()[:,[1]].astype(int)).sum()
    Overlap.loc[x,'ChromLength_atac1'] = (ATAC_1[x].to_numpy()[:,[2]].astype(int) - ATAC_1[x].to_numpy()[:,[1]].astype(int)).sum()
    Overlap.loc[x,'ChromLength_atac2'] = (ATAC_2[x].to_numpy()[:,[2]].astype(int) - ATAC_2[x].to_numpy()[:,[1]].astype(int)).sum()

  # 计算比例
  Overlap.loc[:,'Overlap_atac1'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac1'].values
  Overlap.loc[:,'Overlap_atac2'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac2'].values
  
  return Overlap


In [ ]:
# 提取关键参数
adata_inner.var.loc[:,'ATAC'] = adata_inner.var.index
adata_inner.var = pd.DataFrame((x.split('_') for x in adata_inner.var['ATAC']) , index = adata_inner.var.index , columns = ['chr','chromStart','chromEnd'])
# 提取关键参数
atac1.var.loc[:,'ATAC'] = atac1.var.index
atac1.var = pd.DataFrame((x.split('_') for x in atac1.var['ATAC']) , index = atac1.var.index , columns = ['chr','chromStart','chromEnd'])
# 提取关键参数
atac2.var.loc[:,'ATAC'] = atac2.var.index
atac2.var = pd.DataFrame((x.split('_') for x in atac2.var['ATAC']) , index = atac2.var.index , columns = ['chr','chromStart','chromEnd'])
atac_var = adata_inner.var
atac_1_var = atac1.var
atac_2_var = atac2.var
chr	= set(atac_var.chr)

# 设置重叠率存储变量
Overlap = pd.DataFrame()
OverlapIndex = Overlap.columns.to_list() + ['ChromLength_atac1','ChromLength_atac2','ChromLength_atac_inner','Overlap_atac1','Overlap_atac2']
Overlap = Overlap.reindex(columns=OverlapIndex, fill_value=0)
  # 重叠率计算函数开始
ATAC_INNER = {}
ATAC_1 = {}
ATAC_2 = {}
for x in chr:
  # 染色体拆分
  test = pd.DataFrame(columns=['Length'])
  ATAC_INNER[x] = test
  ATAC_INNER[x]=atac_var[atac_var.chr == x]
  ATAC_INNER[x].loc[:,'length'] = ''
  test = pd.DataFrame(columns=['Length'])
  ATAC_1[x] = test
  ATAC_1[x]=atac_1_var[atac_1_var.chr == x]
  ATAC_1[x].loc[:,'length'] = ''

  test = pd.DataFrame(columns=['Length'])
  ATAC_2[x] = test
  ATAC_2[x]=atac_2_var[atac_2_var.chr == x]
  ATAC_2[x].loc[:,'length'] = ''

    # 输出标记
  print('start %s' % x)
  
    # 计算宽度
  Overlap.loc[x,'ChromLength_atac_inner'] = (ATAC_INNER[x].to_numpy()[:,[2]].astype(int) - ATAC_INNER[x].to_numpy()[:,[1]].astype(int)).sum()
  Overlap.loc[x,'ChromLength_atac1'] = (ATAC_1[x].to_numpy()[:,[2]].astype(int) - ATAC_1[x].to_numpy()[:,[1]].astype(int)).sum()
  Overlap.loc[x,'ChromLength_atac2'] = (ATAC_2[x].to_numpy()[:,[2]].astype(int) - ATAC_2[x].to_numpy()[:,[1]].astype(int)).sum()

# 计算比例
Overlap.loc[:,'Overlap_atac1'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac1'].values
Overlap.loc[:,'Overlap_atac2'] = Overlap.loc[:,'ChromLength_atac_inner'].values/Overlap.loc[:,'ChromLength_atac2'].values

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


start chr11
start chr4
start chr17
start chr10
start chr3
start chr22
start chr20
start chr1
start chrY
start chr18
start chr8
start chr12
start chr19
start chr15
start chr13
start chrX
start chr2
start chr5
start chr21
start chr6
start chr14
start chr7
start chr16
start chr9


In [ ]:
len(atac1.X.todense()[:,0])

5642

In [ ]:
atac_df1 = atac1.to_df()
atac_df1.loc['sum',:] = atac_df1.sum(axis=0)
atac_df1

name,chr11_192013_192414,chr11_201933_202478,chr11_206292_209868,chr11_235447_237832,chr11_250645_251319,chr11_257229_257749,chr11_266646_270176,chr11_279029_282017,chr11_286735_300701,chr11_303203_304090,...,chrY_11316063_11316405,chrY_11320687_11321592,chrY_11322418_11323623,chrY_11329710_11330101,chrY_56829353_56829976,chrY_56842287_56842416,chrY_56845196_56845419,chrY_56848421_56848510,chrY_56849476_56849566,chrY_56850614_56850943
AAACGAAAGACTCGGA-1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAAAGCACCATT-1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAAAGTTGACAA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAACAACTCCCT-1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAACAATGTAAG-1,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCCGGCGAT-1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTTCTACCCGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTTCTGAAAGA-1,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTTCTTACGGA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
atac_inner_df1 = adata_inner.to_df()
atac_inner_df1.loc['sum',:] = atac_inner_df1.sum(axis=0)
atac_inner_df1

index_new,chr11_192013_192414,chr11_201933_202478,chr11_206292_209868,chr11_235447_237832,chr11_250645_251319,chr11_257229_257749,chr11_266646_270176,chr11_279029_282017,chr11_286735_300701,chr11_303203_304090,...,chr6_170271336_170272721,chr6_170279949_170281123,chr6_170288762_170297604,chr6_170305793_170308166,chr6_170438444_170438693,chr6_170459432_170459892,chr6_170476381_170476805,chr6_170516828_170517452,chr6_170529758_170530238,chr6_170552477_170555032
AAACGAAAGACTCGGA-1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAAAGCACCATT-1,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAAAGTTGACAA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACGAACAACTCCCT-1,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0
AAACGAACAATGTAAG-1,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTGTCAGCTTA-1,0.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTGTGCAAGAC-1,0.0,0.0,4.0,6.0,0.0,0.0,2.0,0.0,4.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTTCACTGATG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTGTTCAGTACAC-1,0.0,0.0,6.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [ ]:
atac_df1 = atac1.to_df()
atac_df1.loc['sum',:] = atac_df1.sum(axis=0)
atac_df1

# 函数极端情况测试-删除chrY_的测试

## 函数在极端情况下的测试-atac1删除chrY的测试

In [ ]:
atac1=anndata.read_h5ad("GSE158398_P2003138.h5ad")
atac2=anndata.read_h5ad("GSE158398_P2003139.h5ad")

In [ ]:
# 删除chrY的测试
atac1.var.loc[:,'ATAC'] = atac1.var.index
df1 = pd.DataFrame((x.split('_') for x in atac1.var['ATAC']) , index = atac1.var.index , columns = ['chr','chromStart','chromEnd'])
atac1.var = df1
atac1 = atac1[:,atac1.var[~atac1.var['chr'].isin(['chrY'])].index]
atac1

View of AnnData object with n_obs × n_vars = 5642 × 76831
    var: 'chr', 'chromStart', 'chromEnd'
    uns: 'omic'

In [ ]:
atac1.var[atac1.var['chr']=='chrY']

,chr,chromStart,chromEnd


In [ ]:
len(set(atac1.var.chr))

23

In [ ]:
pair_index = atac_concat_get_index(atac1,atac2)
pair_index

start chr13_
start chr8_
start chr7_
start chrX_
start chr12_
start chr5_
start chr19_
start chrY_
start chr10_
start chr9_
start chr3_
start chr14_
start chr15_
start chr20_
start chr2_
start chr6_
start chr16_
start chr17_
start chr11_
start chr4_
start chr1_
start chr21_
start chr18_
start chr22_


,overleaf_1,overleaf_2,index_new,index_1,index_2
chr13_19564634_19565291,-0.873672,-0.902007,chr13_19564551_19565398,chr13_19564634_19565291,chr13_19564551_19565398
chr13_19586793_19588055,-0.978648,-0.919573,chr13_19586680_19588085,chr13_19586793_19588055,chr13_19586680_19588085
chr13_19601668_19602454,-0.763689,-0.991354,chr13_19601659_19602700,chr13_19601668_19602454,chr13_19601659_19602700
chr13_19633015_19634676,-0.923523,-0.952532,chr13_19632925_19634821,chr13_19633015_19634676,chr13_19632925_19634821
chr13_19775807_19776771,-0.735089,-0.782912,chr13_19775403_19777264,chr13_19775807_19776771,chr13_19775403_19777264
...,...,...,...,...,...
chr22_50687354_50687524,-0.598465,-0.836317,chr22_50687290_50687681,chr22_50687354_50687524,chr22_50687290_50687681
chr22_50724017_50724402,-0.923261,-1.0,chr22_50724017_50724434,chr22_50724017_50724402,chr22_50724017_50724434
chr22_50729335_50729850,-1.185841,-0.95354,chr22_50729314_50729850,chr22_50729335_50729850,chr22_50729314_50729766
chr22_50732072_50732550,-0.965795,-0.995976,chr22_50732070_50732567,chr22_50732072_50732550,chr22_50732070_50732567


In [ ]:
atac_inner = atac_concat_inner(atac1,atac2,pair_index)
atac_inner

AnnData object with n_obs × n_vars = 12844 × 71266

In [ ]:
atac_inner.var[atac_inner.var.index.str.contains('chrY')]

""
index_new
chrY_11293413_11293819
chrY_11295010_11295898
chrY_11314315_11314551
chrY_11332988_11334246
chrY_56836699_56837049


In [ ]:
atac_outer = atac_concat_outer(atac1,atac2,pair_index)
atac_outer

AnnData object with n_obs × n_vars = 12844 × 93421

In [ ]:
atac_outer.var[atac_outer.var.index.str.contains('chrY')]

""
name
chrY_11293413_11293819
chrY_11295010_11295898
chrY_11314315_11314551
chrY_11332988_11334246
chrY_56836699_56837049


## 函数在极端情况下的测试-atac2删除chrY的测试

In [ ]:
atac1=anndata.read_h5ad("GSE158398_P2003138.h5ad")
atac2=anndata.read_h5ad("GSE158398_P2003139.h5ad")

In [ ]:
# 删除chrY的测试
atac2.var.loc[:,'ATAC'] = atac2.var.index
atac2.var = pd.DataFrame((x.split('_') for x in atac2.var['ATAC']) , index = atac2.var.index , columns = ['chr','chromStart','chromEnd'])
atac2 = atac2[:,atac2.var[~atac2.var['chr'].isin(['chrY'])].index]
atac2

View of AnnData object with n_obs × n_vars = 7202 × 87846
    var: 'chr', 'chromStart', 'chromEnd'
    uns: 'omic'

In [ ]:
atac2.var[atac2.var['chr']=='chrY']

,chr,chromStart,chromEnd


In [ ]:
len(set(atac2.var.chr))

23

In [ ]:
pair_index = atac_concat_get_index(atac1,atac2)
pair_index

start chr13_
start chr8_
start chr7_
start chrX_
start chr12_
start chr5_
start chr19_
start chrY_
start chr10_
start chr9_
start chr3_
start chr14_
start chr15_
start chr20_
start chr2_
start chr6_
start chr16_
start chr17_
start chr11_
start chr4_
start chr1_
start chr21_
start chr18_
start chr22_


,overleaf_1,overleaf_2,index_new,index_1,index_2
chr13_19564551_19565398,-1.162861,-1.126332,chr13_19564551_19565398,chr13_19564634_19565291,chr13_19564551_19565398
chr13_19586680_19588085,-1.023772,-1.08954,chr13_19586680_19588085,chr13_19586793_19588055,chr13_19586680_19588085
chr13_19601659_19602700,-1.312977,-1.01145,chr13_19601659_19602700,chr13_19601668_19602454,chr13_19601659_19602700
chr13_19632925_19634821,-1.087297,-1.054184,chr13_19632925_19634821,chr13_19633015_19634676,chr13_19632925_19634821
chr13_19775403_19777264,-1.511411,-1.419087,chr13_19775403_19777264,chr13_19775807_19776771,chr13_19775403_19777264
...,...,...,...,...,...
chr22_50687290_50687681,-1.923529,-1.376471,chr22_50687290_50687681,chr22_50687354_50687524,chr22_50687290_50687681
chr22_50724017_50724434,-1.083117,-1.0,chr22_50724017_50724434,chr22_50724017_50724402,chr22_50724017_50724434
chr22_50729314_50729766,-0.836893,-1.040777,chr22_50729314_50729850,chr22_50729335_50729850,chr22_50729314_50729766
chr22_50732070_50732567,-1.035565,-1.004184,chr22_50732070_50732567,chr22_50732072_50732550,chr22_50732070_50732567


In [ ]:
atac_inner = atac_concat_inner(atac1,atac2,pair_index)
atac_inner

AnnData object with n_obs × n_vars = 12844 × 71288

In [ ]:
atac_inner.var

""
index_new
chr10_100009037_100010704
chr10_100013810_100014868
chr10_100020394_100021040
chr10_100045949_100046542
chr10_10007975_10008392
...
chrY_56842287_56842416
chrY_56845196_56845419
chrY_56848421_56848510


In [ ]:
atac_inner.var[atac_inner.var.index.str.contains('chrY')]

""
index_new
chrY_11292168_11296066
chrY_11305143_11305542
chrY_11306266_11306870
chrY_11307467_11308270
chrY_11310115_11310485
chrY_11312483_11312975
chrY_11313601_11315122
chrY_11316063_11316405
chrY_11320687_11321592


In [ ]:
atac_outer = atac_concat_outer(atac1,atac2,pair_index)
atac_outer

AnnData object with n_obs × n_vars = 12844 × 93427

In [ ]:
atac_outer.var[atac_outer.var.index.str.contains('chrY')]

""
name
chrY_11292168_11296066
chrY_11305143_11305542
chrY_11306266_11306870
chrY_11307467_11308270
chrY_11310115_11310485
chrY_11312483_11312975
chrY_11313601_11315122
chrY_11316063_11316405
chrY_11320687_11321592
